In [1]:
# --- Imports ---
import json
import os
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

C:\Users\sushm\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# --- Config ---
model_name = "BAAI/bge-large-en-v1.5"  # Better quality embedding model
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\ProgramData\anaconda3\envs\llms\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sushm\.cache\huggingface\hub\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:

# --- Files to merge ---
input_files = [
    "ML-data.json",
    "CSE-data.json",
    "cyber-data.json",
    "extra.json"]
# --- Step 1: Merge all projects ---
merged_data = []
for file in input_files:
    if os.path.exists(file):
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
            merged_data.extend(data)
    else:
        print(f"⚠️ File not found: {file}")

print(f"✅ Total projects loaded: {len(merged_data)}")




✅ Total projects loaded: 101


In [7]:
# --- Step 2: Generate embeddings ---
embedded_projects = []

for project in tqdm(merged_data, desc="Generating embeddings"):
    # Combine all important fields into a single text
    combined_text = (
        f"Title: {project.get('title', '')}\n"
        f"Description: {project.get('description', '')}\n"
        f"Objective: {project.get('objective', '')}\n"
        f"Tech Stack: {project.get('tech_stack', '')}\n"
        f"Domain: {project.get('domain', '')}"
    )
    
    # Generate embedding
    embedding = model.encode(combined_text, show_progress_bar=False).tolist()
    
    # Attach embedding to project
    project["embedding"] = embedding
    
    embedded_projects.append(project)

Generating embeddings:   0%|                                                                   | 0/101 [00:00<?, ?it/s]C:\ProgramData\anaconda3\envs\llms\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Generating embeddings: 100%|█████████████████████████████████████████████████████████| 101/101 [00:05<00:00, 16.94it/s]


In [8]:
# --- Step 3: Save final unified file ---
with open("projects-embedded.json", "w", encoding="utf-8") as f:
    json.dump(embedded_projects, f, indent=2)

print(f"✅ Saved unified RAG dataset with embeddings: projects-embedded.json")

✅ Saved unified RAG dataset with embeddings: projects-embedded.json
